## 🔢 บทที่ 2: การคำนวณข้อมูล (Calculation)
---

ในงาน ISP การคำนวณที่ป๋าจะได้ใช้บ่อยที่สุดคือการหา **"ส่วนต่าง"** ไม่ว่าจะเป็นส่วนต่างของราคา หรือส่วนต่างของเวลา (อายุสัญญา)

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Timedelta** | ไทม์-เดลต้า | **ผลต่างของเวลา** (เป็นหน่วย วัน, ชั่วโมง, นาที) |
| **New Column** | นิว คอลัมน์ | การสร้างหัวข้อใหม่ในตารางเพื่อเก็บค่าที่คำนวณได้ |
| **Arithmetic** | อะ-ริธ-เม-ติก | การบวก ลบ คูณ หาร ข้อมูลในตาราง |

> **สูตรสร้างคอลัมน์ใหม่:** `df['ชื่อคอลัมน์ใหม่'] = ค่าที่คำนวณ`

In [8]:
# setup
import pandas as pd
from datetime import datetime

exec_file = pd.ExcelFile('mock_isp_data.xlsx')
sheets_names = exec_file.sheet_names
df = exec_file.parse(sheets_names[0], dtype={'MSISDN': str})
date_cols = ['CONTRACT_START_DT', 'CONTRACT_END_DT']
df[date_cols] = df[date_cols].apply(pd.to_datetime)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   MSISDN             5 non-null      object        
 1   RC_RATE            5 non-null      int64         
 2   SUBS_STATUS        5 non-null      object        
 3   CUST_FULL_NAME     5 non-null      object        
 4   PRODUCT_TYPE       5 non-null      object        
 5   CONTRACT_START_DT  5 non-null      datetime64[ns]
 6   CONTRACT_END_DT    5 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 412.0+ bytes


,MSISDN,RC_RATE,SUBS_STATUS,CUST_FULL_NAME,PRODUCT_TYPE,CONTRACT_START_DT,CONTRACT_END_DT
0,0960673105,599,Active,ABC Corp,FTTH,2023-01-01,2024-01-01
1,0912345678,899,Active,"XYZ Co.,Ltd",Mobile,2023-05-15,2024-05-15
2,0812345678,1200,Suspended,Pah Nattapong Service,Mobile,2022-10-20,2023-10-20
3,0612345678,449,Active,JukThong Tech,DATA SERVICE,2024-01-10,2025-01-10
4,0998887776,299,Terminated,Sample Client,FTTH,2023-12-01,2024-12-01


In [13]:
# 1. กำหนด "วันนี้"
today = pd.Timestamp(datetime.now().date())

# 2. คำนวณหาจำนวนวันที่เหลือ (Contract Duration)
# เราลบกันตรงๆ แล้วดึง .days ออกมาเพื่อเอาเฉพาะตัวเลขวัน
df['REMAIN_DAYS'] = (df['CONTRACT_END_DT'] - today).dt.days

# 3. ลองคำนวณ "มูลค่าสัญญาที่เหลือ" (แบบง่ายๆ)
# สมมติ: วันที่เหลือ x (ราคาต่อเดือน / 30 วัน)
df['REMAIN_VALUE'] = df['REMAIN_DAYS'] * (df['RC_RATE'] / 30)

# 4. โชว์ผลลัพธ์แบบป๋าเลือกเฉพาะคอลัมน์สำคัญ
df[['CUST_FULL_NAME', 'REMAIN_DAYS', 'REMAIN_VALUE']].head()



,CUST_FULL_NAME,REMAIN_DAYS,REMAIN_VALUE
0,ABC Corp,-773,-15434.233333
1,"XYZ Co.,Ltd",-638,-19118.733333
2,Pah Nattapong Service,-846,-33840.000000
3,JukThong Tech,-398,-5956.733333
4,Sample Client,-438,-4365.400000


### ⚠️ การจัดการค่าติดลบ (Expired Contracts)
ถ้าผลการคำนวณออกมาเป็น **ติดลบ (Negative)** แสดงว่าสัญญานั้น **"หมดอายุแล้ว"** ครับ

* ถ้าเป็น `+`: คือจำนวนวันที่เหลือ
* ถ้าเป็น `-`: คือจำนวนวันที่เกินกำหนดมาแล้ว

> **ทริคของจุกทอง:** ป๋าสามารถใช้การกรอง (Filtering) ที่เราเรียนไปบทก่อน มาหาว่าใครบ้างที่สัญญาติดลบ เพื่อส่งทีม Sales ไปคุยได้ทันทีครับ!

In [22]:
expired_list = df[df['REMAIN_DAYS'] < 0]

print(f'มีญญาที่หลดอายุแล้วทั้งหมด {len(expired_list)} รายการ')
display(expired_list[['CUST_FULL_NAME', 'MSISDN', 'REMAIN_DAYS']])

มีญญาที่หลดอายุแล้วทั้งหมด 5 รายการ


,CUST_FULL_NAME,MSISDN,REMAIN_DAYS
0,ABC Corp,0960673105,-773
1,"XYZ Co.,Ltd",0912345678,-638
2,Pah Nattapong Service,0812345678,-846
3,JukThong Tech,0612345678,-398
4,Sample Client,0998887776,-438


### 🚦 การแบ่งกลุ่มลูกค้าตามผลการคำนวณ
เมื่อป๋าได้คอลัมน์ `REMAIN_DAYS` มาแล้ว ป๋าสามารถแบ่งกลุ่มลูกค้าได้ทันที เช่น:
* `REMAIN_DAYS < 0` : **Expired** (หมดอายุแล้ว)
* `0 <= REMAIN_DAYS <= 30` : **Warning** (กำลังจะหมดใน 1 เดือน)
* `REMAIN_DAYS > 30` : **Normal** (ปกติ)

เราจะใช้สิ่งที่เรียกว่า **"Boolean Masking"** (การกรองด้วยเงื่อนไข) เพื่อแยกคนกลุ่มนี้ออกมาครับ

In [16]:
warning_list = df[(df['REMAIN_DAYS'] >= 0) & (df['REMAIN_DAYS'] <= 30)]

print(f'สาขาที่ไกล้หมดสัญญาแล้ว {len(warning_list)} รายการ')
warning_list[['CUST_FULL_NAME', 'MSISDN', 'REMAIN_DAYS']]

สาขาที่ไกล้หมดสัญญาแล้ว 0 รายการ


,CUST_FULL_NAME,MSISDN,REMAIN_DAYS
